Using 'DeepFashion' Dataset for Test Purposes: (Category and Attribute Prediction Benchmark)

- https://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html


DISC2021 Image Similarity Dataset:
- https://sites.google.com/view/isc2021/dataset

*1- Load and Preprocess Images*

In [ ]:
import boto3
import json
import base64
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image

image_dir = "test_images_design_subset"

# Load and preprocess images
def load_images(image_dir):
    image_list = []
    file_names = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(image_dir, filename)
            with open(img_path, "rb") as img_file:
                img_data = img_file.read()
                image_list.append(img_data)
            file_names.append(img_path)
    return image_list, file_names

image_data_list, file_names = load_images(image_dir)

# Display number of loaded images
print(f"Loaded {len(image_data_list)} images.")

*AWS Config*

In [ ]:
import json
import logging
import boto3
import base64

boto3_session = boto3.session.Session(region_name="us-east-1", profile_name="aihubdev")
bedrock_runtime = boto3_session.client(service_name='bedrock-runtime')

model_id = "amazon.titan-embed-image-v1"

accept = "application/json"
content_type = "application/json"

Getting text embedding:

In [ ]:
input_text = "What are the different services that you offer?"
output_embedding_length = 256

body = json.dumps({
    "inputText": input_text,
    "embeddingConfig": {
        "outputEmbeddingLength": output_embedding_length
    }
})

response = bedrock_runtime.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=content_type
)

response_body = json.loads(response.get('body').read())

Getting image embedding:

In [ ]:
with open("test_images/design_test1.png", "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

body = json.dumps({
    "inputImage": input_image,
    "embeddingConfig": {
        "outputEmbeddingLength": output_embedding_length
    }
})

response = bedrock_runtime.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=content_type
)

response_body = json.loads(response.get('body').read())

In [ ]:
response_body

Function to get image embeddings from the Titan Model

In [ ]:
# Function to get image embedding using Titan model
def get_image_embedding_titan(image_data, model_id, accept, content_type, output_embedding_length):
    try:
        input_image = base64.b64encode(image_data).decode('utf8')
        body = json.dumps({
            "inputImage": input_image,
            "embeddingConfig": {
                "outputEmbeddingLength": output_embedding_length
            }
        })
        response = bedrock_runtime.invoke_model(
            body=body, modelId=model_id, accept=accept, contentType=content_type
        )
        response_body = json.loads(response['body'].read().decode('utf8'))
        embedding = response_body['embedding']
        return embedding
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

*2- Extract embeddings using Titan*

In [ ]:
# Get embeddings for all images
embeddings = []
for img_data in image_data_list:
    embedding = get_image_embedding_titan(img_data, model_id, accept, content_type, output_embedding_length)
    if embedding is not None:
        embeddings.append(embedding)

# Convert embeddings to numpy array
embeddings_array = np.array(embeddings)

# Display number of generated embeddings
print(f"Generated {len(embeddings_array)} embeddings.")

*3- Calculate Embedding Similarity*

In [ ]:
# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings_array)
print("Cosine similarity matrix:\n", similarity_matrix)

*4- Do Visualization*

In [ ]:
# Function to visualize using t-SNE with images
def visualize_embeddings_with_images(embeddings, file_names, image_dir):
    tsne = TSNE(n_components=2, perplexity=15, n_iter=300)
    tsne_results = tsne.fit_transform(embeddings)
    
    fig, ax = plt.subplots(figsize=(16, 16))
    for i, filename in enumerate(file_names):
        x, y = tsne_results[i, :]
        #img_path = os.path.join(image_dir, filename)
        img_path = filename
        img = Image.open(img_path)
        img.thumbnail((50, 50), Image.LANCZOS)
        img = np.array(img)
        if img.shape[2] == 4:  # Convert RGBA to RGB if necessary
            img = img[:, :, :3]
        img_box = OffsetImage(img, zoom=1)
        ab = AnnotationBbox(img_box, (x, y), frameon=False)
        ax.add_artist(ab)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0)  # To set the limits of the plot
    plt.title('t-SNE Visualization of Image Embeddings with Images')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

# Visualize the embeddings
visualize_embeddings_with_images(embeddings_array, file_names, image_dir)